<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# NPA: Neural News Recommendation with Personalized Attention
NPA \[1\] is a news recommendation model with personalized attention. The core of NPA is a news representation model and a user representation model. In the news representation model we use a CNN network to learn hidden representations of news articles based on their titles. In the user representation model we learn the representations of users based on the representations of their clicked news articles. In addition, a word-level and a news-level personalized attention are used to capture different informativeness for different users.

## Properties of NPA:
- NPA is a content-based news recommendation method.
- It uses a CNN network to learn news representation. And it learns user representations from their clicked news articles.
- A word-level personalized attention is used to help NPA attend to important words for different users.
- A news-level personalized attention is used to help NPA attend to important historical clicked news for different users.

## Data format:
For quicker training and evaluaiton, we sample MINDdemo dataset from [MIND small dataset](https://msnews.github.io/). The MINDdemo dataset have the same file format as MINDsmall and MINDlarge. If you want to try experiments on MINDsmall
 and MINDlarge, please change the dowload source.
 
**MINDdemo_train** is used for training, and **MINDdemo_dev** is used for evaluation. Training data and evaluation data are composed of a news file and a behaviors file. You can find more detailed data description in [MIND repo](https://github.com/msnews/msnews.github.io/blob/master/assets/doc/introduction.md)

### news data
This file contains news information including newsid, category, subcatgory, news title, news abstarct, news url and entities in news title, entities in news abstarct.
One simple example: <br>

`N46466	lifestyle	lifestyleroyals	The Brands Queen Elizabeth, Prince Charles, and Prince Philip Swear By	Shop the notebooks, jackets, and more that the royals can't live without.	https://www.msn.com/en-us/lifestyle/lifestyleroyals/the-brands-queen-elizabeth,-prince-charles,-and-prince-philip-swear-by/ss-AAGH0ET?ocid=chopendata	[{"Label": "Prince Philip, Duke of Edinburgh", "Type": "P", "WikidataId": "Q80976", "Confidence": 1.0, "OccurrenceOffsets": [48], "SurfaceForms": ["Prince Philip"]}, {"Label": "Charles, Prince of Wales", "Type": "P", "WikidataId": "Q43274", "Confidence": 1.0, "OccurrenceOffsets": [28], "SurfaceForms": ["Prince Charles"]}, {"Label": "Elizabeth II", "Type": "P", "WikidataId": "Q9682", "Confidence": 0.97, "OccurrenceOffsets": [11], "SurfaceForms": ["Queen Elizabeth"]}]	[]`
<br>

In general, each line in data file represents information of one piece of news: <br>

`[News ID] [Category] [Subcategory] [News Title] [News Abstrct] [News Url] [] [ClickedNews0:w1,w2,w3,...] ...`

<br>

We generate a word_dict file to tranform words in news title to word indexes, and a embedding matrix is initted from pretrained glove embeddings.

### behaviors data
One simple example: <br>
`1	U82271	11/11/2019 3:28:58 PM	N3130 N11621 N12917 N4574 N12140 N9748	N13390-0 N7180-0 N20785-0 N6937-0 N15776-0 N25810-0 N20820-0 N6885-0 N27294-0 N18835-0 N16945-0 N7410-0 N23967-0 N22679-0 N20532-0 N26651-0 N22078-0 N4098-0 N16473-0 N13841-0 N15660-0 N25787-0 N2315-0 N1615-0 N9087-0 N23880-0 N3600-0 N24479-0 N22882-0 N26308-0 N13594-0 N2220-0 N28356-0 N17083-0 N21415-0 N18671-0 N9440-0 N17759-0 N10861-0 N21830-0 N8064-0 N5675-0 N15037-0 N26154-0 N15368-1 N481-0 N3256-0 N20663-0 N23940-0 N7654-0 N10729-0 N7090-0 N23596-0 N15901-0 N16348-0 N13645-0 N8124-0 N20094-0 N27774-0 N23011-0 N14832-0 N15971-0 N27729-0 N2167-0 N11186-0 N18390-0 N21328-0 N10992-0 N20122-0 N1958-0 N2004-0 N26156-0 N17632-0 N26146-0 N17322-0 N18403-0 N17397-0 N18215-0 N14475-0 N9781-0 N17958-0 N3370-0 N1127-0 N15525-0 N12657-0 N10537-0 N18224-0`
<br>

In general, each line in data file represents one instance of an impression. The format is like: <br>

`[Impression ID] [User ID] [Impression Time] [User Click History] [Impression News]`

<br>

User Click History is the user historical clicked news before Impression Time. Impression News is the displayed news in an impression, which format is:<br>

`[News ID 1]-[label1] ... [News ID n]-[labeln]`

<br>
Label represents whether the news is clicked by the user. All information of news in User Click History and Impression News can be found in news data file.

## Global settings and imports

In [1]:
import sys
sys.path.append("../../")
from reco_utils.recommender.deeprec.deeprec_utils import download_deeprec_resources 
from reco_utils.recommender.newsrec.newsrec_utils import prepare_hparams
from reco_utils.recommender.newsrec.models.npa import NPAModel
from reco_utils.recommender.newsrec.io.mind_iterator import MINDIterator
import papermill as pm
from tempfile import TemporaryDirectory
import tensorflow as tf
import os

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

tmpdir = TemporaryDirectory()

System version: 3.6.10 |Anaconda, Inc.| (default, May  8 2020, 02:54:21) 
[GCC 7.3.0]
Tensorflow version: 1.15.2


## Download and load data

In [2]:
data_path = "/home/v-jinyi/data/MIND/demo"

train_news_file = os.path.join(data_path, 'train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'train', r'behaviors.tsv')
valid_news_file = os.path.join(data_path, 'valid', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'valid', r'behaviors.tsv')
wordEmb_file = os.path.join("/home/v-jinyi/", "msrec_util", "embedding.npy")
userDict_file = os.path.join("/home/v-jinyi/", "msrec_util", "uid2index.pkl")
wordDict_file = os.path.join("/home/v-jinyi/", "msrec_util", "word_dict.pkl")
yaml_file = os.path.join("/home/v-jinyi/", "msrec_util", r'npa.yaml')

if not os.path.exists(train_news_file):
    download_deeprec_resources(r'https://mind201910small.blob.core.windows.net/release/', \
                               os.path.join(data_path, 'train'), 'MINDsmall_train.zip')
if not os.path.exists(valid_news_file):
    download_deeprec_resources(r'https://mind201910small.blob.core.windows.net/release/', \
                               os.path.join(data_path, 'valid'), 'MINDsmall_dev.zip')

In [3]:
data_path

'/home/v-jinyi/data/MIND/demo'

## Create hyper-parameters

In [4]:
epochs=6
seed=42

In [5]:
hparams = prepare_hparams(yaml_file, wordEmb_file=wordEmb_file, \
                          wordDict_file=wordDict_file, userDict_file=userDict_file,\
                          epochs=epochs)
print(hparams)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

data_format=news,iterator_type=None,support_quick_scoring=False,wordEmb_file=/home/v-jinyi/msrec_util/embedding.npy,wordDict_file=/home/v-jinyi/msrec_util/word_dict.pkl,userDict_file=/home/v-jinyi/msrec_util/uid2index.pkl,vertDict_file=None,subvertDict_file=None,title_size=10,body_size=None,word_emb_dim=300,word_size=None,user_num=None,vert_num=None,subvert_num=None,his_size=50,npratio=4,dropout=0.2,attention_hidden_dim=200,head_num=4,head_dim=100,cnn_activation=relu,dense_activation=None,filter_num=400,window_size=3,vert_emb_dim=100,subvert_emb_dim=100,gru_unit=400,type=ini,user_emb_dim=100,learning_rate=0.0001,loss=cross_entro

In [6]:
iterator = MINDIterator

## Train the NPA model

In [7]:
model = NPAModel(hparams, iterator, seed=seed)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [8]:
print(model.run_eval(valid_news_file, valid_behaviors_file))

{'group_auc': 0.5229, 'mean_mrr': 0.2329, 'ndcg@5': 0.2377, 'ndcg@10': 0.3031}


In [9]:
model.fit(train_news_file, train_behaviors_file, valid_news_file, valid_behaviors_file)

at epoch 1
train info: logloss loss:1.5034317792285972
eval info: group_auc:0.5861, mean_mrr:0.255, ndcg@10:0.3417, ndcg@5:0.2775
at epoch 1 , train time: 33.7 eval time: 65.8
at epoch 2
train info: logloss loss:1.403599177307797
eval info: group_auc:0.6032, mean_mrr:0.272, ndcg@10:0.3623, ndcg@5:0.2982
at epoch 2 , train time: 30.0 eval time: 65.4
at epoch 3
train info: logloss loss:1.3498227250191472
eval info: group_auc:0.6003, mean_mrr:0.2715, ndcg@10:0.3616, ndcg@5:0.2954
at epoch 3 , train time: 30.0 eval time: 65.4
at epoch 4
train info: logloss loss:1.3073714194759245
eval info: group_auc:0.5981, mean_mrr:0.2731, ndcg@10:0.3603, ndcg@5:0.2969
at epoch 4 , train time: 30.0 eval time: 65.5
at epoch 5
train info: logloss loss:1.2704716553885815
eval info: group_auc:0.5958, mean_mrr:0.2733, ndcg@10:0.3594, ndcg@5:0.2942
at epoch 5 , train time: 30.0 eval time: 65.5
at epoch 6
train info: logloss loss:1.237624950892365
eval info: group_auc:0.6035, mean_mrr:0.2765, ndcg@10:0.3637, nd

In [10]:
res_syn = model.run_eval(valid_news_file, valid_behaviors_file)
print(res_syn)
pm.record("res_syn", res_syn)

{'group_auc': 0.6035, 'mean_mrr': 0.2765, 'ndcg@5': 0.2977, 'ndcg@10': 0.3637}


/data/anaconda/envs/reco_gpu/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Function record is deprecated and will be removed in verison 1.0.0 (current version 0.19.1). Please see `scrapbook.glue` (nteract-scrapbook) as a replacement for this functionality.
  This is separate from the ipykernel package so we can avoid doing imports until


## Reference
\[1\] Chuhan Wu, Fangzhao Wu, Mingxiao An, Jianqiang Huang, Yongfeng Huang and Xing Xie: NPA: Neural News Recommendation with Personalized Attention, KDD 2019, ADS track.<br>
\[2\] Wu, Fangzhao, et al. "MIND: A Large-scale Dataset for News Recommendation" Proceedings of the 58th Annual Meeting of the Association for Computational Linguistics. https://msnews.github.io/competition.html <br>
\[3\] GloVe: Global Vectors for Word Representation. https://nlp.stanford.edu/projects/glove/